# Foundations of Data Mining: Assignment Resit NN

Please complete all assignments in this notebook. You should submit this notebook, as well as a PDF version (See File > Download as).

## Fully convolutional neural networks

The goal of this excercise is to develop a model for recognizing seqences of MNIST digits in an image. You are given a image of size $(28 by 28*n)$ where $n$ is also an input parameter. The model needs to be able to recognize the sequence of digits in the image. 

Training many different models for different values of $n$ is not very efficient. A more efficient strategy is to develop a model for detecting single digits and apply that model on the input image by sliding it (to cover the larger surface) and then post processing its output. Sliding a CNN model over a large image in the general case results in re-computing many of the computations. One approach that deals with this efficiently is converting the CNN model into a fully convolutional neural network(*).

Any Dense layer can be converted to a Convolutional layer. For example, a Dense layer with K=128 neurons that inputs an activation map 7×7×512 can be equivalently expressed as a Convolutional layer with filter size of 7x7, (padding = 0, stride 1x1) and K=128 number of filters. Rather than flattening the activation map and feeding it into a dense layer we are using a Convolutional layer and setting the filter size to be exactly the size of the input volume (For more detais: https://arxiv.org/pdf/1411.4038.pdf).

To solve this excercise extend/modify the code given bellow to:
* Produce a CNN model for single digit detection of MNIST images
* Convert this model into a fully convolutional neural network (FCNN)
* Run the FCNN model on a image that is a concatenation of a sequence of MNIST images of length n=5
* Post-process the FCNN output from the previous point to produce a numerical sequence of digits as output
* Discuss how your implementation would need to be modified if the input is not a concatenation of MNIST images but rather the MNIST images are pasted on a random location on a large canvas (no implementatation is necessary)

(*) FCNN models are very useful for processing very large images, such as produced in digital pathology or satelite imaging. This models are applicable to any domain where the size of the objects in the image is much smaller than the image itself. 

In [1]:
## Imports
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv2D, MaxPooling2D, Convolution2D
from keras.layers import Input
from keras import backend as K
import numpy as np
import random
# Training parameters
batch_size = 128
num_classes = 10
epochs = 1

# Data preparation

# input image dimensions
img_rows, img_cols = 28, 28
num_digits = 5

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
    input_shape_test = (1, img_rows, img_cols*num_digits)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    input_shape_test = (img_rows, img_cols * num_digits, 1)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [2]:
# CNN Model definition
def build_cnn_model():

    model = Sequential()
    # <<<<<<<<<<<<<<<<<<
    # Add convolutional layers here to finalize the CNN model
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # <<<<<<<<<<<<<<<<<<
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model


In [3]:
# FCNN Model definition
def build_full_conv_model():

    model = Sequential()
    # <<<<<<<<<<<<<<<<<<<
    # Add convolutional layers here that match the CNN architecture
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape_test))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    # <<<<<<<<<<<<<<<<<<<
    
    # This is are two example Conv layers that do the same operations 
    # as the Dense layers with 128 neurons and the final classification layer as given in the CNN model 
    # replace the <q> value of the size of the filters, such that it matches the dimension of the activation map
    model.add(Conv2D(128, kernel_size=(12, 12),strides=14, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Conv2D(num_classes, kernel_size=(1, 1)))
    model.add(Reshape((-1, num_classes)))
    model.add(Activation("softmax"))

    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])
    return model

In [4]:
def transplant_weight(weights_in, weights_out):
    for layer in range(len(weights_in)):
        weights_out[layer].flat = weights_in[layer].flat
    return weights_out

In [5]:
def build_concat_mnist(images, labels, size=100, length=5):
    # form the appropriate tensor for the result
    image_width = images.shape[2]
    out = np.zeros((size, images.shape[1], image_width*length, images.shape[3]))
    out_labels = np.zeros((size, length, labels.shape[1]))
    # randomly sample images (size x length samples)
    indexes = np.random.randint(0, images.shape[0], (size, length))
    # fill in the final tensor for images and labels
    for i in range(size):
        for j in range(length):
            start = image_width*j
            end = image_width*(j+1)
            out[i, :, start:end] = images[indexes[i, j]]
            out_labels[i, j] = labels[indexes[i, j]]
    return out, out_labels

In [6]:
# build test data
(x_concat_test, y_concat_test) = build_concat_mnist(x_test, y_test)

# build models
model = build_cnn_model()
full_conv_model = build_full_conv_model()

# model file
model_file = "mnist-model-{epoch:02d}-{val_loss:.2f}.hdf5"

# train
model.fit(x_train, y_train,batch_size=batch_size, epochs=epochs, verbose=1,
          validation_data=(x_test, y_test))
#model = train_cnn_model(model_file, model)
# During testing you can load the CNN parameters, rather than train each time
#model_load = "mnist-model.hdf5"
#model.load_weights(model_load)

model_weights = model.get_weights()

full_conv_weights = full_conv_model.get_weights()
full_conv_weights = transplant_weight(model_weights, full_conv_weights)
full_conv_model.set_weights(full_conv_weights)

# Execute the FCNN
output = full_conv_model.predict(x_concat_test, batch_size=batch_size)
# The output of the fully conv model needs to be processed to produce the sequence of digits
output = np.argmax(output, axis=2)
print("Model output")
# <<<<<<<<<<<<<<<
# post processing the model output
print(output)
# <<<<<<<<<<<<<<<


# the true labels for comparison
labels = np.argmax(y_concat_test, axis=2)
print("Label outputs")
print(labels)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 102s 2ms/step - loss: 0.2755 - acc: 0.9145 - val_loss: 0.0597 - val_acc: 0.9809
Model output
[[3 8 7 2 7]
 [1 5 5 6 0]
 [8 1 2 3 1]
 [9 9 2 4 1]
 [5 2 9 9 7]
 [3 0 6 4 1]
 [9 4 9 6 1]
 [9 2 7 7 2]
 [4 2 3 2 6]
 [1 3 9 4 8]
 [8 8 2 4 3]
 [2 6 8 1 4]
 [6 3 4 0 0]
 [8 1 5 7 4]
 [2 8 7 5 3]
 [5 7 1 1 3]
 [4 3 4 7 6]
 [2 8 0 7 4]
 [1 4 1 7 5]
 [2 3 7 6 6]
 [9 3 0 6 7]
 [7 0 9 0 0]
 [2 2 1 2 6]
 [2 4 6 4 6]
 [2 7 4 7 2]
 [2 2 2 7 7]
 [5 1 9 9 2]
 [3 4 2 1 2]
 [9 5 9 1 5]
 [7 9 5 3 7]
 [6 3 5 1 9]
 [0 1 5 4 3]
 [0 9 2 8 1]
 [8 2 3 7 9]
 [6 4 6 2 6]
 [7 6 8 2 8]
 [3 0 0 3 8]
 [6 1 7 6 8]
 [9 7 2 8 2]
 [1 4 3 3 8]
 [1 6 0 8 7]
 [7 2 2 6 6]
 [5 3 8 6 3]
 [1 0 

In [10]:
count = 0
for i in range(0,100):
    for j in range(0,5):
        if(output[i][j]==labels[i][j]):
            count = count+1
            
accuracy = count/500
print(accuracy)

0.976
